# Finding fiducial magnitudes for $t_{\mbox{eff}}$

## Introduction

$t_{\mbox{eff}}$ is a measure of depth, related to limiting magnitude as:

\begin{align}
t_{\mbox{eff}} & = 10^{\frac{4}{5}(m_\circ - m_{\mbox{lim}})} \\
m_{\mbox{lim}} & = m_\circ + 1.25 \log t_{\mbox{eff}}
\end{align}

such limiting magnitude varies with $t_{\mbox{eff}}$ in the same way as it varies with exposure time.

The constant $m_\circ$ provides a reference point, defining at what magnitude $t_{\mbox{eff}} = 1$, analogous to the constant reference magnitude in the definition of a magnitude system: accumulated $t_{\mbox{eff}}$ represents the progress made toward reaching a target coadd limiting magnitude, as measured in exposure time under constant conditions. If $m_\circ$ is the typical limiting magnitude for the instrument under real conditions, the difference between the accumulated $t_{\mbox{eff}}$ and the $t_{\mbox{eff}}$ corresponding to the target limiting magnitude is a good approximation of the total exposure time needed to attain the target limiting magnitude. If $m_\circ$ is different from that typical for the instrument, then the remaining $t_{\mbox{eff}}$ will be related to the remaining needed exposure time by a scaling factor. 

A reasonable fiducial magnitude for a given instrument is to define for $t_{\mbox{eff}}=1$ for optimally pointing images under typical conditions: taken at zenith, during dark time, in typical seeing. With this fiducial magnitude, the remaining needed $t_{\mbox{eff}}$ matches the remaining exposure time if optimally scheduled, and the target limiting magnitude varies with the zenith distance at transit.

`opsim` provides a function, `m5_flat_sed`, to calculate the limiting magnitude (including various throughputs, etc.) under a given set of conditions (seeing, airmass, and sky brightness) using parameters provided by system engineering.

The initial section of this notebook uses the mean log Fried parameter (according to the model described in RTN-022) to deteremine the seeing, the sky brightness model used by `opsim` in full dark conditions, zenith pointing (airmass=1), and `m5_flat_sed` to compute fiducial zero points for $t_{\mbox{eff}}$.

The later section of the notebook generates magnitude using different assumptions about the throughput, seeing, sky brightness, and airmass to trace the differences between magnitudes derived from the SRD and estimates based on the latest assumptions used by `opsim`.

## Notebook setup

### Logging

Do it first so I can use logging to track how long imports take.

In [1]:
import logging
logging.basicConfig(format='%(asctime)s %(message)s')
logger = logging.getLogger(__name__)
logger.setLevel('DEBUG')
logger.info("Starting")

2021-07-07 10:16:27,876 Starting


### Imports

In [2]:
import os
import os.path
import copy
import pickle
import urllib
from os import path
from tempfile import TemporaryDirectory

In [3]:
logger.debug("Loading common modules")
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3

2021-07-07 10:16:27,911 Loading common modules


In [4]:
logger.debug("Loading general astronomy modules")
import healpy
import palpy
import astropy
import astropy.coordinates
from astropy.coordinates import EarthLocation
from astropy.time import Time
import astropy.units as u

2021-07-07 10:16:28,736 Loading general astronomy modules


In [5]:
logger.debug("Loading Rubin Observatory modules")
import rubin_sim
import rubin_sim.site_models
from rubin_sim.utils import m5_flat_sed
from rubin_sim.site_models import SeeingData, SeeingModel
from rubin_sim.site_models import CloudData
from rubin_sim.skybrightness_pre import SkyModelPre

2021-07-07 10:16:29,385 Loading Rubin Observatory modules


### Plot setup

In [6]:
mpl.rcParams['figure.figsize'] = (8, 5)
plt.style.use('ggplot')
np.random.seed(6563)

### Constants

In [7]:
sky_model = SkyModelPre()
site = EarthLocation.of_site('Cerro Pachon')
mean_log_r0 = -0.9163 ;# See RTN-022, table 1
mjd_sample_rate = 0.01 ;# Every 0.01 days = every 14.4 minutes
original_median_seeing500 = 0.62
data_dir = '.'
opsim_origin = 'https://lsst.ncsa.illinois.edu/sim-data/sims_featureScheduler_runs1.7/baseline/baseline_nexp2_v1.7_10yrs.db'

In [8]:
bands = pd.DataFrame({
    'band': ('u', 'g', 'r', 'i', 'z', 'y'),
    'min_wavelength': (350.0, 400.0, 552.0, 691.0, 818.0, 949.0),
    'max_wavelength': (400.0, 552.0, 691.2, 818.0, 922.0, 1060.0)}
).set_index('band', drop=False)
bands

,band,min_wavelength,max_wavelength
band,,,
u,u,350.0,400.0
g,g,400.0,552.0
r,r,552.0,691.2
i,i,691.0,818.0
z,z,818.0,922.0
y,y,949.0,1060.0


In [9]:
sun_cache_fname = 'sun_cache.pickle'
moon_cache_fname = 'moon_cache.pickle'
sky_cache_fname = 'sky_cache.pickle'
sky_x12_cache_fname = 'sky_x12_cache.pickle'
visit_cache = 'opsim_visits.h5'

### Retrieving data

In [10]:
if path.isfile(opsim_origin) or 'opsim_fname' in locals():
    # Check for opsim_fname in locals so that if we rerun the notebook without restarting the kernel
    # it will not re-download the data
    opsim_fname = opsim_origin
else:
    opsim_data_dir = TemporaryDirectory(dir=data_dir)
    opsim_fname = path.join(opsim_data_dir.name, opsim_origin.split('/')[-1])
    logger.info("Downloading %s to %s", opsim_origin, opsim_fname)
    opsim_db_file = urllib.request.urlretrieve(opsim_origin, opsim_fname)
    logger.info("Download complete")
    
    print(f"Opsim data file downloaded to\n\t{opsim_fname}\nwhich will be deleted when the notebook is restarted or closed.")

NameError: name 'path' is not defined

In [ ]:
opsim_name = os.path.splitext(os.path.split(opsim_fname)[-1])[0]

### Configuration

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Seeing

The seeing model of RTN-022 provides a mean log r0 (Fried parameter), which provides an average seeing. Begin by transforming that to a von Karman seeing with an outer scale of 30m:

In [ ]:
def vk_seeing(r0, outer_scale=30.0, wavelength=5.0e-7):
    """Calculate the seeing using a von Karman model.

    See Tokovinin 2002PASP..114.1156T

    Args:
        r0: the Fried parameter, in meters
        outer_scale: the von Karman outer scale, in meters
        wavelength: the wavelength of light, in meters

    Returns:
        The PSF FWHM, in arcseconds
    """
    # Calculate the DIMM estimate of the seeing using the Kolmogorov model,
    # using eqn 5 from Tokovinin 2002PASP..114.1156T eqn 5
    kol_seeing = 0.98*wavelength/r0

    # Calculate the correction factor required to convert the Kolmogorov model
    # seeing to the von Karman model seeing,
    # using eqn 19 of Tokovinin 2002PASP..114.1156T
    vk_correction2 = 1.0 - 2.183*np.power(r0/outer_scale, 0.356)

    # Apply the correction factor
    seeing_rad = kol_seeing * np.sqrt(vk_correction2)

    # Convert to arcseconds
    seeing = np.degrees(seeing_rad)*(60.0*60.0)

    return seeing

seeing500 = np.round(vk_seeing(10**mean_log_r0, outer_scale=30), 2)
seeing500

Now, apply the `opsim` seeing model:

In [ ]:
seeing_model = SeeingModel()
seeing = pd.Series(seeing_model(seeing500, 1.0)['fwhmEff'], index=bands.band)
seeing

## Sky

Sample the sky at intervals for all dark time in the survey:

In [ ]:
%%time
mjd_range = (astropy.time.Time('2022-10-01T16:00:00Z').mjd, astropy.time.Time('2032-10-01T16:00:00Z').mjd)
all_mjds = np.arange(*mjd_range, mjd_sample_rate)
all_times = astropy.time.Time(all_mjds, scale='utc', format='mjd', location=site)

try:
    with open(sun_cache_fname, 'rb') as sun_cache:
        sun_coords = pickle.load(sun_cache)
    with open(moon_cache_fname, 'rb') as moon_cache:
        moon_coords = pickle.load(moon_cache)
except FileNotFoundError:
    sun_coords = astropy.coordinates.get_sun(all_times).transform_to(astropy.coordinates.AltAz(obstime=all_times, location=site))
    with open(sun_cache_fname, 'wb') as fp:
        pickle.dump(sun_coords, fp)

    moon_coords = astropy.coordinates.get_moon(all_times).transform_to(astropy.coordinates.AltAz(obstime=all_times, location=site))
    with open(moon_cache_fname, 'wb') as moon_cache:
        pickle.dump(moon_coords, moon_cache)

dark_idxs = np.logical_and(sun_coords.alt.deg < -18, moon_coords.alt.deg < -18)
mjds = all_mjds[dark_idxs]
times = all_times[dark_idxs]

Find the healpixels for the standard pointing (zenith) for the sampled times:

In [ ]:
def find_zenith_hpixs(mjd, site, nside, north=True):
    lst = Time(mjd, format='mjd', location=site).sidereal_time('apparent').to_value(u.degree)
    ra = lst
    decl = site.lat.deg
    hpix = healpy.ang2pix(nside, ra, decl, lonlat=True)
    return hpix

Actually calculate the sky brightnesses:

In [ ]:
def compute_zenith_mags(mjds, sky_model, **kwargs):
    zenith_hpixs = pd.DataFrame({'mjd': mjds,
                                 'hpix': find_zenith_hpixs(mjds, **kwargs)})
    mags = zenith_hpixs.apply(
        lambda x: pd.Series({k: m[0] for k, m in sky_model.returnMags(x.mjd, indx=np.array([x.hpix], dtype=int), badval=np.nan, zenith_mask=False).items()}),
        axis=1)
    mags['mjd'] = mjds
    return mags
    
if __debug__:
    df = compute_zenith_mags(np.arange(60000.7, 60001.7, 0.01), sky_model, site=site, nside=sky_model.nside)
    ax = None
    for band in bands.band:
        ax = df.plot('mjd', band, ax=ax)

In [ ]:
%%time
try:
    with open(sky_cache_fname, 'rb') as sky_cache:
        dark_sky_mags = pickle.load(sky_cache)
except FileNotFoundError:
    dark_sky_mags = compute_zenith_mags(mjds, sky_model=sky_model, site=site, nside=sky_model.nside)
    with open(sky_cache_fname, 'wb') as sky_cache:
        pickle.dump(dark_sky_mags, sky_cache)

Get the median dark sky in each band:

In [ ]:
median_dark_sky = dark_sky_mags.median()
median_dark_sky

## Calculate the fiducial magnitudes

In [ ]:
try:
    # Make sure we are using parameters as defined in the code. 
    delattr(m5_flat_sed, 'Cm')
except AttributeError:
    pass

m0 = bands.copy()
m0.apply(
        lambda r: m5_flat_sed(r['band'], median_dark_sky[r['band']], seeing[r['band']], expTime=15, airmass=1, nexp=2), axis=1).round(2)

# Tracing the origin of differences with the SRD

## SRD m5

Copy values from the SRD:

In [ ]:
m5 = pd.DataFrame(
    {'SRD spec': [23.9, 25.0, 24.7, 24.0, 23.3, 22.1],
     'SRD min': [23.4 ,24.6, 24.3, 23.6, 22.9, 21.7],
     'SRD stretch': [24.0, 25.1, 24.8, 24.1, 23.4, 22.2]},
    index=bands.band)

## Reproduce m5 from the overview paper

Table 2 from p. 26 of the [overview paper](https://arxiv.org/abs/0805.2366).

In [ ]:
overview_table2 = pd.DataFrame({'m_sky': [22.99, 22.26, 21.20, 20.48, 19.60, 18.61],
                                'theta': [0.81, 0.77, 0.73, 0.71, 0.69, 0.68],
                                'theta_eff': [0.92, 0.87, 0.83, 0.80, 0.79, 0.76],
                                'gamma': [0.038, 0.039, 0.039, 0.039, 0.039, 0.039],
                                'k_m': [0.491, 0.213, 0.126, 0.096, 0.069, 0.170],
                                'C_m': [23.09, 24.42, 24.44, 24.32, 24.16, 23.73],
                                'm5': [23.78, 24.81, 24.35, 23.92, 23.34, 22.45],
                                'Delta_C_m_infinity': [0.62, 0.18, 0.10, 0.07, 0.05, 0.04],
                                'Delta_C_m_2': [0.23, 0.08, 0.05, 0.03, 0.02, 0.02],
                                'Delta_m5': [0.21, 0.16, 0.14, 0.13, 0.13, 0.14],
                               }, index=bands.index)

overview_table2.T

Call `m5_flat_sed` once to instantiate default parameters.

In [ ]:
try:
    # Make sure we are using parameters as defined in the code. 
    delattr(m5_flat_sed, 'Cm')
except AttributeError:
    pass

_ = m5_flat_sed('u', 23, 1.0, 30.0, 1.0)

In [ ]:
m5_flat_sed.Cm = overview_table2.C_m.to_dict()
m5_flat_sed.dCm_infinity = overview_table2.Delta_C_m_infinity.to_dict()
m5_flat_sed.kAtm = overview_table2.k_m
m5_flat_sed.msky = overview_table2.m_sky

In [ ]:
m5['overview'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'], r['m_sky'], r['theta_eff'], expTime=30, airmass=1.0, nexp=1), axis=1).values
m5

Compare m5 as re-derived from the table using `m5_flat_sed` with the actual values from the table:

In [ ]:
m5['overview'] - overview_table2['m5']

Okay, so close, but maybe some rounding difference.

## Move to airmass of 1.2

See if we can reproduce the `theta_eff` from overview table 2 using `SeeingModel`, so we can use it to calculate the effective seeing at other airmasses.

In [ ]:
seeing_model = SeeingModel()

In [ ]:
test_eff_seeing = pd.DataFrame(
    {'table 2': overview_table2.theta_eff.values,
     'SeeingModel': seeing_model(original_median_seeing500, 1.0)['fwhmEff']},
index=bands.index)
test_eff_seeing['difference'] = test_eff_seeing['SeeingModel'] - test_eff_seeing['table 2']
test_eff_seeing

Pre-compute the effective seeing at airmass

In [ ]:
original_seeing_12 = pd.Series(seeing_model(original_median_seeing500, 1.2)['fwhmEff'], index=bands.index)
original_seeing_12

In [ ]:
m5['airmass'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'], r['m_sky'], original_seeing_12[r['band']], expTime=30, airmass=1.2, nexp=1), axis=1).values
m5

Compare change to airmass 1.2 to that reported in table 2:

In [ ]:
test_airmass12 = pd.DataFrame({'table 2 Delta_m5': overview_table2.Delta_m5,
                               'Delta_m5': m5['overview'] - m5['airmass']})
test_airmass12['diff'] = test_airmass12['Delta_m5'] - test_airmass12['table 2 Delta_m5']
test_airmass12

## Update to use throughput in `sims_utils`

Trigger reinitialzation of the parameters in `m5_flat_sed` to be defaults when called:

In [ ]:
del m5_flat_sed.Cm

In [ ]:
m5['throughput'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'], r['m_sky'], original_seeing_12[r['band']], expTime=30, airmass=1.2, nexp=1), axis=1).values
m5

## Switch to two exposures per visit

In [ ]:
m5['2exp'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'], r['m_sky'], original_seeing_12[r['band']], expTime=15, airmass=1.2, nexp=2), axis=1).values
m5

Compare with the difference reported in overview table 2:

In [ ]:
pd.DataFrame({'calculated': m5['throughput'] - m5['2exp'],
              'overview': overview_table2['Delta_C_m_2'],
              'diff': overview_table2['Delta_C_m_2'] + m5['2exp'] - m5['throughput']})

## Change to the latest seeing model

In [ ]:
seeing_airmass12 = pd.Series(seeing_model(seeing500, 1.2)['fwhmEff'], index=bands.band)
seeing_airmass12

In [ ]:
m5['seeing'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'], r['m_sky'], seeing_airmass12[r['band']], expTime=15, airmass=1.2, nexp=2), axis=1).values
m5

## Change to the latest sky brightness model and airmass=1.2

In [ ]:
night_idxs = sun_coords.alt.deg < -14
night_mjds = all_mjds[night_idxs]
night_times = all_times[night_idxs]

In [ ]:
def compute_zd_from_airmass(airmass, a=470.0):
    mu = (1.0/airmass) + (1/(2.0*a))*((1.0/airmass) - airmass)
    zd = np.degrees(np.arccos(mu))
    return zd

if __debug__:
    # Bemporad values (empirical values for airmass as a function of zenith distance)
    assert compute_zd_from_airmass(1)==0
    assert np.round(compute_zd_from_airmass(1.154), 1)==30
    assert np.round(compute_zd_from_airmass(1.995), 1)==60
    assert np.round(compute_zd_from_airmass(2.904), 1)==70
    assert np.round(compute_zd_from_airmass(18.8))==88

In [ ]:
def compute_transiting_hpixs(mjd, airmass, site, nside, north=True):
    zd = compute_zd_from_airmass(airmass)
    lst = Time(mjd, format='mjd', location=site).sidereal_time('apparent').to_value(u.degree)
    ra = lst
    decl = site.lat.deg + zd if north else site.lat.degrees - zd
    hpix = healpy.ang2pix(nside, ra, decl, lonlat=True)
    return hpix

Actually calculate the sky brightnesses:

In [ ]:
def compute_transiting_mags(mjds, sky_model, **kwargs):
    transiting_hpixs = pd.DataFrame({'mjd': mjds,
                                     'hpix': compute_transiting_hpixs(mjds, **kwargs)})
    mags = transiting_hpixs.apply(
        lambda x: pd.Series({k: m[0] for k, m in sky_model.returnMags(x.mjd, indx=np.array([x.hpix], dtype=int), badval=np.nan, zenith_mask=False).items()}),
        axis=1)
    mags['mjd'] = mjds
    return mags
    
if __debug__:
    df = compute_transiting_mags(np.arange(60000.7, 60001.7, 0.01), sky_model, airmass=1.2, site=site, nside=sky_model.nside)
    ax = None
    for band in bands.band:
        ax = df.plot('mjd', band, ax=ax)

In [ ]:
%%time
try:
    with open(sky_x12_cache_fname, 'rb') as sky_cache:
        sky_mags_x12 = pickle.load(sky_cache)
except FileNotFoundError:
    sky_mags_x12 = compute_transiting_mags(night_mjds, sky_model=sky_model, airmass=1.2, site=site, nside=sky_model.nside)
    with open(sky_x12_cache_fname, 'wb') as sky_cache:
        pickle.dump(sky_mags_x12, sky_cache)

In [ ]:
sky_x12_mag_stats = sky_mags_x12.describe()
sky_x12_mag_stats

The nominal total number of u, g, and r exposures is the same as the nominal total z and y.

If we assume u, g, and r are observed in the darkest 50% of time, the z and y in the brightest, and the i evenly split, then we get:

In [ ]:
typical_m_sky = pd.Series([
    sky_x12_mag_stats.loc['75%', 'u'],
    sky_x12_mag_stats.loc['75%', 'g'],
    sky_x12_mag_stats.loc['75%', 'r'],
    sky_x12_mag_stats.loc['50%', 'i'],
    sky_x12_mag_stats.loc['25%', 'z'],
    sky_x12_mag_stats.loc['25%', 'y']],
    index=bands.index
)
typical_m_sky

In [ ]:
m5['sky'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'], typical_m_sky[r['band']], seeing_airmass12[r['band']], expTime=15, airmass=1.2, nexp=2), axis=1).values
m5

## Baseline median and mean

Look at what a baseline `opsim` simulation actually achieves.

In [ ]:
try:
    visits = pd.read_hdf(visit_cache, opsim_name)
except (FileNotFoundError, KeyError):
    with sqlite3.connect(opsim_fname) as con:
        visits = pd.read_sql_query('SELECT * FROM SummaryAllProps', con)
    visits.set_index('filter', drop=False, inplace=True)
    visits.index.name = 'band'
    visits.to_hdf(visit_cache, opsim_name)

In [ ]:
m5['b. median'] = visits.groupby('band').agg({'fiveSigmaDepth': 'median'}).loc[bands.index, 'fiveSigmaDepth'].T
m5['b. mean'] = visits.groupby('band').agg({'fiveSigmaDepth': 'mean'}).loc[bands.index, 'fiveSigmaDepth'].T
m5

## Baseline mag from mean t_eff

In [ ]:
def compute_teff(ref_mags, mags=visits.fiveSigmaDepth):
    t_eff = 10**(0.8*(mags-ref_mags.loc[mags.index]))
    return t_eff

In [ ]:
m5['b. teff mean'] = m5['SRD spec'] + 1.25*np.log10(compute_teff(m5['SRD spec'], visits.fiveSigmaDepth).groupby('band').mean())
m5

In [ ]:
print(m5)

## Plot how the magnitude limits change with each modification from the SRD

In [ ]:
m5_imp = m5[['overview', 'airmass', 'throughput', '2exp', 'seeing', 'sky']].T
m5_imp.index.rename('step', inplace=True)
m5_imp.reset_index(inplace=True)
fig, axes = plt.subplots(2, 3, figsize=(16, 12))
for band, ax in zip(bands.band, axes.flatten()):
    ax.plot(m5_imp.index, m5_imp[band], marker='o', linestyle=' ', color='darkblue')
    ax.plot(m5_imp.index, m5_imp[band], alpha=0.2, color='darkblue')
    ax.set_xticks(np.arange(len(m5_imp['step'])))
    ax.set_xticklabels(m5_imp['step'], rotation=20)
    ax.set_title(band)
    ax.axhline(m5.loc[band, 'SRD spec'], linestyle=':', color='k')
    ax.text(0, m5.loc[band, 'SRD spec'], 'SRD spec', horizontalalignment='left', verticalalignment='top', color='k')
    ax.axhline(m5.loc[band, 'SRD min'], linestyle=':', color='r')
    ax.text(0, m5.loc[band, 'SRD min'], 'SRD min', horizontalalignment='left', verticalalignment='bottom', color='r')
    ax.axhline(m5.loc[band, 'SRD stretch'], linestyle=':', color='g')
    ax.text(0, m5.loc[band, 'SRD stretch'], 'SRD stretch', horizontalalignment='left', verticalalignment='top', color='g')
    ax.axhline(m5.loc[band, 'b. median'], linestyle='--', color='b')
    ax.text(0, m5.loc[band, 'b. median'], 'baseline 1.7 sim median', horizontalalignment='left', verticalalignment='top', color='b')
    ax.axhline(m5.loc[band, 'b. teff mean'], linestyle='--', color='k')
    ax.text(0, m5.loc[band, 'b. teff mean'], 'baseline 1.7 sim t_eff mean', horizontalalignment='left', verticalalignment='top', color='k')

## Examine as differences from baseline averages

In [ ]:
m5.apply(lambda c: c - m5['b. teff mean'], axis=0)

In [ ]:
m5.apply(lambda c: c - m5['b. median'], axis=0)

In [ ]:
m5.apply(lambda c: c - m5['b. mean'], axis=0)

## Find t_eff stats

In [ ]:
def compute_teff_stats(*args, **kwargs):
    teff_stats = compute_teff(*args, **kwargs).groupby('band').describe().loc[bands.index].T
    return teff_stats

In [ ]:
m5.apply(lambda r: compute_teff_stats(r).loc['mean'])

In [ ]:
m5.apply(lambda r: compute_teff_stats(r).loc['50%'])

## Progression table

In [ ]:
m5

In [ ]:
m5prog = m5[['overview']].copy()

m5prog['throughput'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'], r['m_sky'], overview_table2['theta_eff'][r['band']], expTime=15, airmass=1, nexp=2), axis=1).values
m5prog['seeing'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'], r['m_sky'],
                              pd.Series(seeing_model(seeing500, 1.0)['fwhmEff'], index=bands.index)[r['band']],
                              expTime=15, airmass=1, nexp=2), axis=1).values
m5prog['sky'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'],
                              dark_sky_mags[r['band']].median(),
                              pd.Series(seeing_model(seeing500, 1.0)['fwhmEff'], index=bands.index)[r['band']],
                              expTime=15, airmass=1, nexp=2), axis=1).values
m5prog['X_k'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'],
                              dark_sky_mags[r['band']].median(),
                              pd.Series(seeing_model(seeing500, 1.0)['fwhmEff'], index=bands.index)[r['band']],
                              expTime=15, airmass=1.2, nexp=2), axis=1).values
m5prog['X_seeing'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'],
                              dark_sky_mags[r['band']].median(),
                              pd.Series(seeing_model(seeing500, 1.2)['fwhmEff'], index=bands.index)[r['band']],
                              expTime=15, airmass=1.2, nexp=2), axis=1).values
m5prog['X'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'],
                              sky_x12_mag_stats.loc['75%', r['band']],
                              pd.Series(seeing_model(seeing500, 1.2)['fwhmEff'], index=bands.index)[r['band']],
                              expTime=15, airmass=1.2, nexp=2), axis=1).values
m5prog['moon'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'],
                              typical_m_sky[r['band']],
                              pd.Series(seeing_model(seeing500, 1.2)['fwhmEff'], index=bands.index)[r['band']],
                              expTime=15, airmass=1.2, nexp=2), axis=1).values
print(m5prog)

In [ ]:
print(m5.loc[:, ['b. median', 'b. mean']].apply(lambda x: x - m5prog.moon).rename(columns={'b. median': 'sim median - model', 'b. mean': 'sim mean - model'}))

In [ ]:
m5p_imp = m5prog.T
m5p_imp.index.rename('step', inplace=True)
m5p_imp.reset_index(inplace=True)
fig, axes = plt.subplots(2, 3, figsize=(16, 12))
for band, ax in zip(bands.band, axes.flatten()):
    ax.plot(m5p_imp.index, m5p_imp[band], marker='o', linestyle=' ', color='darkblue')
    ax.plot(m5p_imp.index, m5p_imp[band], alpha=0.2, color='darkblue')
    ax.set_xticks(np.arange(len(m5p_imp['step'])))
    ax.set_xticklabels(m5p_imp['step'], rotation=20)
    ax.set_title(band)
    ax.hlines(m5.loc[band, ['SRD spec', 'SRD min', 'SRD stretch']], xmin=0, xmax=3, linestyle=':', color='k')
    ax.text(0, m5.loc[band, 'SRD spec'], 'SRD spec', horizontalalignment='left', verticalalignment='top', color='k')
    ax.text(0, m5.loc[band, 'SRD min'], 'SRD min', horizontalalignment='left', verticalalignment='bottom', color='r')
    ax.text(0, m5.loc[band, 'SRD stretch'], 'SRD stretch', horizontalalignment='left', verticalalignment='top', color='g')
    ax.axhline(m5.loc[band, 'b. median'], linestyle='--', color='b')
    ax.text(0, m5.loc[band, 'b. median'], 'baseline 1.7 sim median', horizontalalignment='left', verticalalignment='top', color='b')
    ax.axhline(m5.loc[band, 'b. teff mean'], linestyle='--', color='k')
    ax.text(0, m5.loc[band, 'b. teff mean'], 'baseline 1.7 sim t_eff mean', horizontalalignment='left', verticalalignment='top', color='k')

## Using median airmass, seeing, sky brightness of baseline visits

In [ ]:
m5visits = m5prog[['overview', 'throughput', 'seeing', 'sky']].copy()
m5visits['sim_X'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'],
                              dark_sky_mags[r['band']].median(),
                              pd.Series(seeing_model(seeing500, 1.0)['fwhmEff'], index=bands.index)[r['band']],
                              expTime=15, airmass=visits.loc[visits['filter']==r['band'],'airmass'].median(), nexp=2), axis=1).values
m5visits['sim_seeing'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'],
                              dark_sky_mags[r['band']].median(),
                              visits.loc[visits['filter']==r['band'], 'seeingFwhmEff'].median(),
                              expTime=15, airmass=visits.loc[visits['filter']==r['band'], 'airmass'].median(), nexp=2), axis=1).values
m5visits['sim_sky'] = overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'],
                              visits.loc[visits['filter']==r['band'], 'skyBrightness'].median(),
                              visits.loc[visits['filter']==r['band'], 'seeingFwhmEff'].median(),
                              expTime=15, airmass=visits.loc[visits['filter']==r['band'],'airmass'].median(), nexp=2), axis=1).values
print(m5visits)

In [ ]:
print(m5.loc[:, ['b. median', 'b. mean']].apply(lambda x: x - m5visits.sim_sky).rename(columns={'b. median': 'sim median - model', 'b. mean': 'sim mean - model'}))

In [ ]:
m5p_imp = m5visits.T
m5p_imp.index.rename('step', inplace=True)
m5p_imp.reset_index(inplace=True)
fig, axes = plt.subplots(2, 3, figsize=(16, 12))
for band, ax in zip(bands.band, axes.flatten()):
    ax.plot(m5p_imp.index, m5p_imp[band], marker='o', linestyle=' ', color='darkblue')
    ax.plot(m5p_imp.index, m5p_imp[band], alpha=0.2, color='darkblue')
    ax.set_xticks(np.arange(len(m5p_imp['step'])))
    ax.set_xticklabels(m5p_imp['step'], rotation=20)
    ax.set_title(band)
    ax.hlines(m5.loc[band, ['SRD spec', 'SRD min', 'SRD stretch']], xmin=0, xmax=3, linestyle=':', color='k')
    ax.text(0, m5.loc[band, 'SRD spec'], 'SRD spec', horizontalalignment='left', verticalalignment='top', color='k')
    ax.text(0, m5.loc[band, 'SRD min'], 'SRD min', horizontalalignment='left', verticalalignment='bottom', color='r')
    ax.text(0, m5.loc[band, 'SRD stretch'], 'SRD stretch', horizontalalignment='left', verticalalignment='top', color='g')
    ax.axhline(m5.loc[band, 'b. median'], linestyle='--', color='b')
    ax.text(0, m5.loc[band, 'b. median'], 'baseline 1.7 sim median', horizontalalignment='left', verticalalignment='top', color='b')
    ax.axhline(m5.loc[band, 'b. teff mean'], linestyle='--', color='k')
    ax.text(0, m5.loc[band, 'b. teff mean'], 'baseline 1.7 sim t_eff mean', horizontalalignment='left', verticalalignment='top', color='k')

## Sky brightness and moon

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(13, 8))
for band, ax in zip(bands.band, axes.flatten()):
    visits.query(f"band == '{band}'").hist('skyBrightness', bins=100, ax=ax)
    text_trans = mpl.transforms.blended_transform_factory(ax.transData, ax.transAxes)
    
    ax.axvline(sky_x12_mag_stats.loc['mean', band], color='darkblue')
    ax.text(sky_x12_mag_stats.loc['mean', band], 1, 'mean at X=1.2', transform=text_trans, horizontalalignment='right', verticalalignment='top', rotation=90, color='darkblue')

    ax.axvline(sky_x12_mag_stats.loc['min', band], color='black', linestyle=":")
    ax.text(sky_x12_mag_stats.loc['min', band], 1, 'min at X=1.2', transform=text_trans, horizontalalignment='right', verticalalignment='top', rotation=90, color='black')
    ax.axvline(sky_x12_mag_stats.loc['max', band], color='black', linestyle=":")
    
    ax.axvline(sky_x12_mag_stats.loc['25%', band], color='black', linestyle='--')
    ax.text(sky_x12_mag_stats.loc['25%', band], 1, '25% at X=1.2', transform=text_trans, horizontalalignment='right', verticalalignment='top', rotation=90, color='black')
    ax.axvline(sky_x12_mag_stats.loc['75%', band], color='black', linestyle='--')

    ax.axvline(sky_x12_mag_stats.loc['50%', band], color='black')
    ax.text(sky_x12_mag_stats.loc['50%', band], 1, 'median at X=1.2', transform=text_trans, horizontalalignment='right', verticalalignment='top', rotation=90, color='black')

    ax.set_title(band)

In [ ]:
sky_mags_x12.describe()

## Checking parameters from sims_utils

In [ ]:
sims_utils_params = pd.DataFrame({'C_m': m5_flat_sed.Cm,
                                  'Delta_C_m_infinity': m5_flat_sed.dCm_infinity})
param_compare = pd.concat([overview_table2[sims_utils_params.columns], sims_utils_params], keys=['overview table 2', 'sims_utils'], axis=1).reorder_levels((1,0), axis=1)
param_compare = param_compare.reindex(sorted(param_compare.columns), axis=1)
print(param_compare)

## Coadd-depth based $t_{\mbox{eff}}$

In [ ]:
coadd_depth = pd.DataFrame({'mag': [26.1, 27.4, 27.5, 26.8, 26.1, 24.9],
                            'visits': [56, 80, 184, 184, 160, 160]}, index=bands.band).reindex(visits.index)
coadd_depth['m0'] = coadd_depth.mag - 1.25*np.log10(coadd_depth.visits)
coadd_depth['prog'] = m5prog.moon
coadd_depth['round'] = np.round(m5prog.sky*2)/2
coadd_depth['m0_visit_teff'] = (10**(0.8*(visits.fiveSigmaDepth - coadd_depth.m0))).groupby('band').mean()
coadd_depth['prog_visit_teff'] = (10**(0.8*(visits.fiveSigmaDepth - coadd_depth.prog))).groupby('band').mean()
coadd_depth['round_visit_teff'] = (10**(0.8*(visits.fiveSigmaDepth - coadd_depth['round']))).groupby('band').mean()
coadd_depth.groupby('band').first().loc[bands.band]

## Best possible as reference for $t_{\mbox{eff}}$

In [ ]:
overview_table2.reset_index().apply(
        lambda r: m5_flat_sed(r['band'],
                              dark_sky_mags[r['band']].max(),
                              pd.Series(seeing_model(0.0, 1.0)['fwhmEff'], index=bands.index)[r['band']],
                              expTime=15, airmass=1, nexp=2), axis=1)

In [ ]:
best_m5 = pd.DataFrame({
    'm5': overview_table2.reset_index().apply(lambda r: m5_flat_sed(r['band'],
                                                                    dark_sky_mags[r['band']].max(),
                                                                    pd.Series(seeing_model(0.0, 1.0)['fwhmEff'], index=bands.index)[r['band']],
                                                                    expTime=15, airmass=1, nexp=2), axis=1).values},
    index=bands.band)
best_m5['round_best'] = np.round(best_m5.m5*2)/2
best_m5['m5 mean teff'] = 30*(10**(0.8*(visits.fiveSigmaDepth - best_m5.reindex(visits.index).m5))).groupby('band').mean()
best_m5['round mean teff'] = 30*(10**(0.8*(visits.fiveSigmaDepth - best_m5.reindex(visits.index).round_best))).groupby('band').mean()
best_m5

### Distributions of t_eff in units of ideal visits, unrounded reference points

In [ ]:
teff = (10**(0.8*(visits.fiveSigmaDepth - best_m5.reindex(visits.index).m5)))
fig, axes = plt.subplots(3,2, sharex=True, sharey=True)
for band, ax in zip(bands.band, axes.flatten()):
    teff[band].hist(bins=np.arange(20)/30, ax=ax, density=True)
    ax.text(0.97, 0.95, band, horizontalalignment='right', verticalalignment='top', transform=ax.transAxes)
    ax.set_yticklabels([])
plt.tight_layout()

### Distributions in units of ideal seconds, unrounded reference points

In [ ]:
teff = 30*(10**(0.8*(visits.fiveSigmaDepth - best_m5.reindex(visits.index).m5)))
fig, axes = plt.subplots(3,2, sharex=True, sharey=True)
for band, ax in zip(bands.band, axes.flatten()):
    teff[band].hist(bins=np.arange(20), ax=ax, density=True)
    ax.text(0.97, 0.95, band, horizontalalignment='right', verticalalignment='top', transform=ax.transAxes)
    ax.set_yticklabels([])
plt.tight_layout()

### Distributions in unites of ideal seconds, rounded reference points

In [ ]:
round_teff = 30*(10**(0.8*(visits.fiveSigmaDepth - best_m5.reindex(visits.index).round_best)))

In [ ]:
fig, axes = plt.subplots(3,2, sharex=True, sharey=True)
for band, ax in zip(bands.band, axes.flatten()):
    round_teff[band].hist(bins=np.arange(20), ax=ax, density=True)
    ax.text(0.97, 0.95, band, horizontalalignment='right', verticalalignment='top', transform=ax.transAxes)
    ax.set_yticklabels([])
plt.tight_layout()